DNN과 다르게 RNN에서는 okt형태소 토큰화 후 워드임베딩을 통해 문장의 문맥을 파악하는 방식으로 학습 시킬것이다. 
일반적으로 불용어를 제거=> 노이즈를 줄여서 성능향상을 도모
그러나 감성분류 5가지에선 불용어 제거하지않고 형태소를 분리하고 어절을 그대로 사용한다.

In [2]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
train_text = open(train_file,'rb').read().decode(encoding='utf-8')


In [3]:
import pandas as pd

df_train = pd.DataFrame({
    'feature':[row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0],
    'target': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})


In [4]:
#데이터 크기 조절
df_train= df_train[::4]

In [5]:
#문장 추출
texts= [ ]
for i in df_train['feature']:
    texts.append(i)
len(texts)

37500

In [6]:
#문자열이 아닌게 있는지 확인
for i in texts:
    if type(i)!=str:
        print(i)

In [7]:
# train 데이터의 입력(X)에 대한 정제(Cleaning)
import re
from soynlp.normalizer import repeat_normalize

def clean_korean_text(text):
    # 특수 문자 및 숫자 제거
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)
    # 반복되는 자음, 모음 제거 (e.g., 'ㅋㅋㅋ' -> 'ㅋ')
    text = repeat_normalize(text, num_repeats=1)
    # 띄어쓰기 정규화 (연속된 공백 문자를 하나의 공백 문자로 변환)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

clean_texts=[]
for i in texts:
    clean_texts.append(clean_korean_text(i))

In [8]:
len(clean_texts)

37500

In [9]:
#Mecab 토큰화
from konlpy.tag import Mecab
mecab=Mecab(dicpath=r'C:/mecab/mecab-ko-dic')

tokenized_text = [ mecab.morphs(i) for i in clean_texts]

In [10]:
len(tokenized_text)

37500

In [11]:
#데이터 불균형 확인
print(df_train.iloc[:,1][df_train.iloc[:,1]==0].count())
print(df_train.iloc[:,1][df_train.iloc[:,1]==1].count())

18678
18822


In [12]:
#y값 추출, 데이터 차원 맞추기, 넘파이 배열로 변환
y_train=(df_train.iloc[:,1].to_frame()).to_numpy()

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#tokenized_text의 빈도수 상위 만개 어절을 토크나이저에 fit시킴
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(tokenized_text)

#단어 빈도수 시각화
#불용어제거후 시각화해볼수도있음

#시퀀스데이터로 변환 ==벡터화
sequences = tokenizer.texts_to_sequences(tokenized_text)

In [14]:
#길이를 맞춰주는 패딩작업,
x_train = pad_sequences(sequences, maxlen=40, padding='post')

In [15]:
#시퀀스 길이확인
print(x_train[0])
x_train.shape

[ 35  80 943  41 245  22  40 646   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]


(37500, 40)

In [16]:
# LSTM,gru
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding,BatchNormalization,Dropout
from tensorflow.keras.layers import GRU, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Embedding( 10000, 300, input_length=40 ),
    LSTM(32,return_sequences=True),
    GRU(32),
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
optimizer= Adam(learning_rate =0.002)
model.compile(optimizer=optimizer , loss='binary_crossentropy', metrics=['accuracy'])

#콜백정의
ES =EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)

#모델학습
model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.25, callbacks=[ES])


Epoch 1/20
220/220 [==============================] - 36s 125ms/step - loss: 0.5526 - accuracy: 0.6847 - val_loss: 0.5114 - val_accuracy: 0.8095
Epoch 2/20
220/220 [==============================] - 28s 128ms/step - loss: 0.3565 - accuracy: 0.8479 - val_loss: 0.4276 - val_accuracy: 0.8103
Epoch 3/20
220/220 [==============================] - 25s 112ms/step - loss: 0.2741 - accuracy: 0.8886 - val_loss: 0.4691 - val_accuracy: 0.8135
Epoch 4/20
220/220 [==============================] - 25s 112ms/step - loss: 0.2187 - accuracy: 0.9108 - val_loss: 0.5508 - val_accuracy: 0.8158
Epoch 5/20
220/220 [==============================] - 25s 113ms/step - loss: 0.1749 - accuracy: 0.9312 - val_loss: 0.6412 - val_accuracy: 0.8112
Epoch 6/20
220/220 [==============================] - 25s 113ms/step - loss: 0.1454 - accuracy: 0.9451 - val_loss: 0.7282 - val_accuracy: 0.7977


In [17]:
#테스트문장 입력
test = ['회식끝나고 집가는중','개꿀잼이네','진짜 너무 별로다.','ㅋㅋㅋㅋㅋㅋㅋㅋ']

#정제
clean_test_texts=[]
for i in test:
    clean_test_texts.append(clean_korean_text(i))

#mecab
tokenized_test_text = [ mecab.morphs(i) for i in clean_test_texts]

#벡터화
test_sequences = tokenizer.texts_to_sequences(tokenized_test_text) 
x_test = pad_sequences(test_sequences,padding='post',maxlen=40) 

#예측
prediction = model.predict(x_test)  

for i in np.round(prediction):
    print(int(i))

1/1 [==============================] - 1s 1s/step
0
1
0
1


# 카톡대화 입력

In [18]:
# 카톡대화 불러와서 정제,(정규화코드)하는 함수
import re

def get_from_txt(txt):
    data= open(txt,"r", encoding='utf-8').read().split('\n')
    sentences=[]
    for line in data:
        pattern = r'\[(.*?)\]\s+\[(.*?)\]\s+(.+)'
        match = re.match(pattern, line)
        if match:
            name = match.group(1)  # 첫 번째 대괄호 안의 단어 추출
            time = match.group(2)  # 두 번째 대괄호 안의 단어 추출
            content = match.group(3)  # 대괄호 뒤의 내용 추출
            # print(name, time, content)
            temp=[name,time,content]
            sentences.append(temp)    
    return sentences

In [19]:
#닉네임 입력단 
target_name = str(input())
print(target_name)  #김하영 입력

김하영


In [20]:
#입력된 이름의 '대화내역만' 담기
received_texts= []
for i in get_from_txt('sample.txt'): 
    if i[0] == target_name:
        received_texts.append( i[2] )
        
# 이모티콘, 사진, 샵검색 제거 
clean_received_texts1 = []
for i in received_texts:
    if '샵검색:' not in i: 
        if "이모티콘" not in i:
            if '샵검색:' not in i:
                clean_received_texts1.append(str(i))

#=========== 이쯤에서 답장시간 계산기 구현   =======================
          
          
#텍스트 정제  
clean_received_texts2= [clean_korean_text(i) for i in clean_received_texts1]


#문자열이 아닌게 있는지 확인
for i in clean_received_texts2:
    if type(i)!=str:
        print(i)
        
# train 데이터 입력값(X)을 정제(Cleaning)
import re
from soynlp.normalizer import repeat_normalize

def clean_korean_text(text):
    # 특수 문자 및 숫자 제거
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)
    # 반복되는 자음, 모음 제거 (e.g., 'ㅋㅋㅋ' -> 'ㅋ')
    text = repeat_normalize(text, num_repeats=1)
    # 띄어쓰기 정규화 (연속된 공백 문자를 하나의 공백 문자로 변환)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


clean_test_texts=[]
for i in clean_received_texts2:
    clean_test_texts.append(clean_korean_text(i))


#mecab으로 토큰화
tokenized_clean_test_texts= [mecab.morphs(i) for i in clean_test_texts ]


#시퀀스데이터로 변환
test_sequences = tokenizer.texts_to_sequences(tokenized_clean_test_texts) 
paded_test_sequences = pad_sequences(test_sequences,padding='post',maxlen=40) 
prediction = model.predict(paded_test_sequences)  


cnt0=0;cnt1=0

for i in prediction:
    if float(i)>=0.5:
        print('긍정문입니다.')
        cnt1+=1
    elif float(i)<0.5:
        print('부정문입니다.')
        cnt0+=1
        
#부정과 긍정문의 갯수
print('부정문과 긍정문의 갯수:',cnt0,cnt1)

#긍정과 부정의 비율( 긍정문의 수 / 부정문의 수)
print('긍정과 부정의 비율: ',cnt1/cnt0)
#숫자가 1이상이고 높을수록 긍정적이다.

6/6 [==============================] - 0s 15ms/step
긍정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
긍정문입니다.
부정문입니다.
부정문입